In [ ]:
import json
import csv
import numpy as np
import networkx as nx
import math
import pandas as pd
import copy

In [ ]:
threshord_year = 2018

# Your need to load the text and pub_year (abs_year) of your corpus here.

In [ ]:
former_abs = []
later_abs = []

for tuple in abs_year:
    abs = tuple[0]
    year = tuple[1]
    if int(year) <= threshord_year:
        former_abs.append(abs)
    else:
        later_abs.append(abs)

# You need to load the result from ER.ipynb (NERoutput) here.

In [ ]:
trainset = []
evalset = []

for i in NERoutput:
    doi = i[-1]
    year = DOI_Year.get(doi, 'UNKNOWN')
    if year == 'UNKNOWN':
        continue
    elif int(year) <= threshord_year:
        trainset.append(i)
    else:
        evalset.append(i)

In [ ]:
trainset = create_triples(trainset)
trainset = clean_triples(trainset)
evalset = create_triples(evalset)
evalset = clean_triples(evalset)
evalapp = {}

for eval in evalset:
    if eval[2] == 'hasApplication':
        if eval[4] not in evalapp.keys():
            evalapp[eval[4]] = [eval[1]]
        else:
            evalapp[eval[4]].append(eval[1])

# Define G

In [ ]:
G = nx.Graph()

for head_label, head, relation, tail_label, tail in trainset:
    if head not in G:
        G.add_node(head, label=head_label, name=head)
    if tail not in G:
        G.add_node(tail, label=tail_label, name=tail)
    G.add_edge(head, tail, relationship=relation)

    G.add_edge(head, tail, relationship=relation)

# Node Similarity

In [ ]:
def calculate_jaccard_similarity(set1, set2):
    """Calculate the Jaccard similarity between two sets"""
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    
    return len(intersection) / len(union) if union else 0

def jaccard_similarity():
    target_relations = 'hasApplication'
    related_relations = ['hasStructure/Phase', 'hasProperty']
    excluded_relations = ['hasName', 'hasAcronym']
    label = 'Application'
    min_set_size = 5  # Set the minimum set size as the filtering condition
    results = []
    app = []

    for node, attr in G.nodes(data=True):
        if attr.get('label') == 'Application':
            app.append(attr.get('name'))

    for name in app:
        excluded_nodes = set()
        connected_nodes = {}
        material_nodes = {}
        application_prop = {}
        target_prop = set()
        
        target_nodes = [node for node, attr in G.nodes(data=True) if attr.get('label') == label and node == name]
        
        if target_nodes:
            target_node = target_nodes[0]
            for neighbour in G.neighbors(target_node):
                relationship = G[target_node][neighbour].get('relationship')
                if relationship in target_relations:
                    if neighbour not in connected_nodes:
                        connected_nodes[neighbour] = []
        else:
            print("No node found with the specified label and name.")
        
        for node, relationships in list(connected_nodes.items()):
            for neighbour in G.neighbors(node):
                relationship = G[node][neighbour].get('relationship')
                if relationship in related_relations:
                    relationships.append(neighbour)
        
        for node in connected_nodes.keys():
            excluded_nodes.add(node)
            for neighbour in G.neighbors(node):
                relationship = G[node][neighbour].get('relationship')
                if relationship in excluded_relations:
                    excluded_nodes.add(neighbour)
        
        for node, attr in G.nodes(data=True):
            if attr.get('label') in ['Formula'] and node not in excluded_nodes:
                material_nodes[node] = []
                for neighbour in G.neighbors(node):
                    relationship = G[node][neighbour].get('relationship')
                    if relationship in related_relations:
                        material_nodes[node].append(neighbour)
        
        for neighbour in G.neighbors(target_node):
            relationship = G[target_node][neighbour].get('relationship')
            if relationship == 'hasProperty':
                target_prop.add(neighbour)
        
        for node, attr in G.nodes(data=True):
            if attr.get('label') == 'Application' and node != name:
                application_prop[node] = []
                for neighbour in G.neighbors(node):
                    relationship = G[node][neighbour].get('relationship')
                    if relationship == 'hasProperty':
                        application_prop[node].append(neighbour)
    
        for mat_node, mat_features in material_nodes.items():
            # Convert material properties into sets
            mat_features_set = set(mat_features)
            if len(mat_features_set) < min_set_size:
                continue
        
            max_material_similarity = 0
            most_similar_connected_material = None
            material_similarity_scores = []
        
            # Traverse the materials in connected nodes and calculate similarity
            for connected_node, connected_features in connected_nodes.items():
                connected_features_set = set(connected_features)
                if len(connected_features_set) >= min_set_size:
                    similarity_score = calculate_jaccard_similarity(mat_features_set, connected_features_set)
                    material_similarity_scores.append(similarity_score)
                    
                    # Update maximum similarity and most similar known materials
                    if similarity_score > max_material_similarity:
                        max_material_similarity = similarity_score
                        most_similar_connected_material = connected_node
        
            app_similarity_score = 0
            for neighbour in G.neighbors(mat_node):
                relationship = G[mat_node][neighbour].get('relationship')
                if relationship == 'hasApplication':
                    if neighbour in application_prop:
                        app_features_set = set(application_prop[neighbour])
                        current_app_similarity = calculate_jaccard_similarity(app_features_set, target_prop)
                        app_similarity_score = max(app_similarity_score, current_app_similarity)
        
            # Calculate the final similarity as the product of material similarity and application similarity
            final_similarity = max_material_similarity * app_similarity_score
            if final_similarity > 0:
                results.append((mat_node, most_similar_connected_material, final_similarity, name))
        
        results.sort(key=lambda x: x[2], reverse=True)
    return results


jresults = jaccard_similarity()

In [178]:
pred = set()

for result in jresults[:200]:
    isright = False
    mat = result[0]
    score = result[2]
    app = result[3]
    app_sep = result[3].split()
    for abs in former_abs:
        if mat in abs and all(word in abs for word in app_sep):
            isright = True
            break
    if isright != True and app not in ['solar cell', 'water splitting'] and len(mat) > 3:
        pred.add((mat, app, score))

sorted_pred = sorted(pred, key=lambda x: x[2], reverse=True)